In [24]:
!pip3 install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [25]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from tqdm import tqdm
import wandb,os
PROJECT_NAME = 'Intel-Image-Classification-Learning-PyTorch-Lightning'

In [26]:
from torch.utils.data import DataLoader,Dataset

In [27]:
idx = 0
labels = {}
labels_r = {}

In [28]:
for label in tqdm(os.listdir('./data/')):
    idx += 1
    labels[label] = idx
    labels_r[idx] = label

100%|█████████████████████████████████████████| 6/6 [00:00<00:00, 113359.57it/s]


In [29]:
class DataSet_Loader(DataLoader):
    def __init__(self):
        super().__init__()
        self.labels = {}
        self.labels_r = {}
        self.idx = 0
        self.data = []
        for label in tqdm(os.listdir('./data/')):
            self.idx += 1
            self.labels[label] = idx
            self.labels_r[idx] = label
        for folder in tqdm(os.listdir('./data/')):
            for file in os.listdir(f'./data/{folder}/'):
                img = cv2.imread(f'./data/{folder}/{file}')
                img = cv2.resize(img,(56,56))
                img = img / 255.0
                self.data.append([
                    img,
                    np.eye(labels[folder],len(labels))[labels[folder]-1]
                ])
        np.random.shuffle(self.data)
        self.X = []
        self.y = []
        for d in self.data:
            self.X.append(d[0])
            self.y.append(d[1])
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,idx):
        return self.X[idx],self.y[idx]

In [30]:
criterion = nn.MSELoss()
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.activation = nn.ReLU()
        self.linear1 = nn.Linear(3*56*56,256)
        self.linear2 = nn.Linear(256,512)
        self.linear3 = nn.Linear(512,1024)
        self.output = nn.Linear(1024,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.output(preds)
        return preds
    
    def training_step(self,batch,batch_idx):
        images,labels = batch
        images = images.view(-1,56*56*3)
        outputs = self(images)
        loss = criterion(outputs,labels)
        wandb.log({'Loss':loss.item()})
        return {'train_loss':loss}
    
    def train_dataloader(self):
        train_data = DataSet_Loader()
        dataset = DataLoader(train_data,batch_size=32,shuffle=True)
        return dataset
    
    def val_dataloader(self):
        train_data = DataSet_Loader()
        dataset = DataLoader(train_data,batch_size=32,shuffle=True)
        return dataset
    
    def validation_step(self, batch,batch_idx):
        images,labels = batch
        images = images.view(-1,56*56*3)
        outputs = self(images)
        loss = criterion(outputs,labels)
        wandb.log({'Val Loss':loss.item()})
        return {'val_loss':loss}
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=0.001)

In [31]:
wandb.init(project=PROJECT_NAME,name='baseline')
model = Model()
trainer = Trainer(max_epochs=125)
trainer.fit(model)
wandb.finish()

wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


AttributeError: 'int' object has no attribute 'log_hyperparams'